In [1]:
##IMPORT LIBRARIES

import googletrans
import pytesseract
import cv2
from googletrans import Translator
import texthero as hero
import pandas as pd
import nltk
import spacy
import random
import time
import numpy as np

import re
import string
from moviepy.editor import *

import sys
from spacy import displacy

from tqdm.auto import tqdm
from spacy.tokens import DocBin

In [4]:
###FUNCTIONS TO PREPROCESS TEXT

def listToString(s):         
    str1 = " "         
    return (str1.join(s))

def cleaner(list):
    series = pd.Series(list)
    return hero.preprocessing.clean(series)

def predict(string):
    return nlp_textcat(string).cats

### LOAD EVERYTHING NECESSARY

nlp_textcat = spacy.load("textcat_output/model-best") #SPACY MODEL

path = 'test.mp4' #PATH TO VIDEO

translator = Translator() #DEFINE TRANSLATOR

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #PATH TO PYTESSERACT APP

clip = VideoFileClip(path) #DEFINE CLIP FOR THE MOVIEPY LIBRARY 


### FUNCTION TO GET THE CATEGORY LIST



def get_cat(path):
    cat_list = []
    words = []
    translated = []
    count = 0
    vidcap = cv2.VideoCapture(path)
    success,image = vidcap.read()
    while success:

        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
        cv2.imwrite("frame%d.jpg" % count, image[825:908,250:1328])#TUNE THE NUMBERS INFRONT OF image TO GET THE TEXT     # save frame as JPEG file ,      
        success,image = vidcap.read()
        print('Read a new frame: ', success)
        path = r'C:\Users\patri\OneDrive\Ambiente de Trabalho\modelo\frame'+str(count)+'.jpg'

        text=pytesseract.image_to_string(path, lang='por')
        text=text.strip()
        print(text)
        if text == '':
            cat_list.append('-')
        else:

            
            
            result = translator.translate(text, src='pt')
            
            cat = max(nlp_textcat(result.text).cats, key=nlp_textcat(result.text).cats.get)
            print(cat)
            if cat == 'sports':
                cat_list.append('Sports : ' + str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
            
            elif cat == 'general':
                cat_list.append('Society/Politics : ' + str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
            else:
                cat_list.append('Technology : ' + str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
            
        
        count += 5 #number of seconds each frame is taken
    
    return cat_list
        

In [4]:
###LABEL A VIDEO AND SAVE TO CHOSEN DIRECTORY
cat_list= []
timer=[]
words = []
translated = []
count = 0
vidcap = cv2.VideoCapture(path)
success,image = vidcap.read()
clip_list= []
while success:

    vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
    cv2.imwrite("frame%d.jpg" % count, image[825:908,250:1328]) #TUNE THE NUMBERS INFRONT OF image TO GET THE TEXT     # save frame as JPEG file      
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    path = r'C:\Users\patri\OneDrive\Ambiente de Trabalho\modelo\frame'+str(count)+'.jpg'

    text=pytesseract.image_to_string(path, lang='por')
    text=text.strip()
    if text == '':
        x='-'
    
    else:
           
        result = translator.translate(text, src='pt')
            
        cat = max(nlp_textcat(result.text).cats, key=nlp_textcat(result.text).cats.get)
            
        if cat == 'sports':
                x='Sports'
                cat_list.append('Sports : ')
                timer.append(str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
        elif cat == 'general':
                x='Society'
                cat_list.append('Society/Politics : ') 
                timer.append(str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
        else:
                x='Technology'
                cat_list.append('Technology : ')
                timer.append(str(int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)))
                
                       
    txt_clip = TextClip(x, fontsize = 30, color = 'white').set_duration(1)
    
    clip_list.append(txt_clip)
    
    count += 5 #number of seconds each frame is taken

datevideo = concatenate(clip_list, method = "compose")

video2 = CompositeVideoClip([clip, datevideo])

##Write video
video2.write_videofile("tagged.mp4",threads = 8,fps=24)
df = pd.DataFrame(np.column_stack([cat_list,timer]), 
                               columns=['Topic','Seconds'])

y = df

y['Topic'] = y['Topic'].drop_duplicates()  ###CAN BE IMPROVED WITH A FOR LOOP

y=y.dropna()

print(y)


KeyboardInterrupt: 